# NLP pour l’analyse de critiques de films

Dans ce brief, il est question d'analyser le sentiment à travers des critiques en français de spectateurs sur des films.

![](https://simplonline.co/_next/image?url=https%3A%2F%2Fsimplonline-v3-prod.s3.eu-west-3.amazonaws.com%2Fmedia%2Fimage%2Fjpg%2F0f6e77dc-ff57-4d34-82a0-8b6a3ac75a03.jpg&w=1280&q=75)

In [1]:
# Montage du drive pour sauver les données (Colab)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
if drive is not None :
  root_path = '/content/drive/MyDrive/ISEN/Projets/NLP-Critiques_Films'
else:
  root_path = "."

root_path

'/content/drive/MyDrive/ISEN/Projets/NLP-Critiques_Films'

## Contexte du projet

En règle générale, le nombre d'avis sur un film peu être important et par conséquent le temps de lecture de chaque commentaire peut être une tâche lourde. Alors comment déterminer de manière rapide si un film a eu du succès auprès des spectateurs (ou pas) ? Dans ce contexte, l’idée du projet est d’utiliser des algorithmes d'apprentissage automatique pour la tâche d'analyse de sentiment des spectateurs via leur critique.

Tout d’abord, il sera question que récupérer les données directement du site d’Allociné. En d’autres termes, nous allons scraper les pages qui nous intéressent sur ce site à savoir les critiques des personnes pour le film Inception et Sonic 2.

En navigant sur la page des critiques, vous vous apercevrez que seules deux types d’information ici nous intéresse : la note du spectateur ainsi que son avis. Pourquoi la note ? Parce que nous allons entraîner un modèle de type supervisé et plus précisément un classifieur et donc la note va nous aider à récupérer la classe pour étiqueter le commentaire. Pour cela, nous considérerons qu’une note au-dessus de 3 est considérée comme satisfaisante. Sinon, l’avis est négatif. Ici, nous avons donc réduit le problème à une classification binaire.

Voici donc les étapes à réaliser :
- Récupération des données
- Préparation des données
- Préparation du modèle et des jeux de données (entrainement & test)
- Analyse des résultats


## Etape 1 : Web Scraping des données d’avis de spectacteurs

De l’avis du spectateur, nous ne devons « scraper » que deux zones la note et le commentaire.

Ressources :
- [critiques du film Sonic 2](https://www.allocine.fr/film/fichefilm-281203/critiques/spectateurs/)
- [critiques du film Inception](https://www.allocine.fr/film/fichefilm-143692/critiques/spectateurs/)


Import des bibliothèques

In [3]:
import requests
import lxml.html as lh
import pandas as pd

import csv

Les films à importer :
- le *titre* va être utilisé pour définir le nom du fichier csv
- l'url est celle de la page de présentation du film sur Allociné

In [4]:
# Films (titre : url)
films = {
    'Sonic2' : 'https://www.allocine.fr/film/fichefilm-281203/critiques/spectateurs/',
    'Inception' : 'http://www.allocine.fr/film/fichefilm-143692/critiques/spectateurs/',
    }


In [5]:
# URI-SUFFIX
uri_pages = '?page='

# Number of pages to read
nbPages = 10

# XPath content to collect
tags = ['//span[@class="stareval-note"]', \
        '//div[@class="content-txt review-card-content"]' ]
cols = ['Note', \
        'Description' ]

In [6]:
def getPage(url):
    """Récupère les critiques de la page de critiques donnée
    """
    print("getPage : ", url)
    page = requests.get(url)
    doc = lh.fromstring(page.content)

    # Get the Web data via XPath
    content = []
    for i in range(len(tags)):
        content.append(doc.xpath(tags[i]))

    # Gather the data into a Pandas DataFrame array
    df_liste = []
    for j in range(len(tags)):
        tmp = pd.DataFrame([content[j][i].text_content().strip() for i in range(len(content[i]))], columns=[cols[j]])
        tmp['key'] = tmp.index
        df_liste.append(tmp)

    # Build the unique Dataframe with one tag (xpath) content per column
    liste = df_liste[0]
    for j in range(len(tags)-1):
        liste = liste.join(df_liste[j+1], on='key', how='left', lsuffix='_l', rsuffix='_r')
        liste['key'] = liste.index
        del liste['key_l']
        del liste['key_r']
    
    return liste

In [7]:
def getPages(_nbPages, _url):
    """ Récupère les critiques de nbpages de pages de critiques
    """
    liste_finale = pd.DataFrame()
    for i in range (_nbPages):
        print("GetPAges : ", i)
        liste = getPage(_url + uri_pages + str(i+1))
        liste_finale = pd.concat([liste_finale, liste], ignore_index=True)
    return liste_finale

In [8]:
def getFilm(titre, url):
  """ Scrappe une page de film et l'enregistre en csv
  """
  file_path = root_path + '/Datasets/allocine_'+ titre +'_avis.csv'
  liste_totale = getPages(nbPages, url)
  print('Save : ', file_path)
  liste_totale.to_csv(file_path, index=False, quoting=csv.QUOTE_NONNUMERIC)

  return liste_totale


In [9]:
nbPages = 1

liste_totale = pd.DataFrame()

for titre, url in films.items() :
  liste_totale = pd.concat([liste_totale, getFilm(titre, url)], ignore_index=True)


GetPAges :  0
getPage :  https://www.allocine.fr/film/fichefilm-281203/critiques/spectateurs/?page=1
Save :  /content/drive/MyDrive/ISEN/Projets/NLP-Critiques_Films/Datasets/allocine_Sonic2_avis.csv
GetPAges :  0
getPage :  http://www.allocine.fr/film/fichefilm-143692/critiques/spectateurs/?page=1
Save :  /content/drive/MyDrive/ISEN/Projets/NLP-Critiques_Films/Datasets/allocine_Inception_avis.csv



## Création du Dataframe



In [10]:
import pandas as pd

In [11]:
def get_dataframe(titre):
  file_path = root_path + '/Datasets/allocine_'+ titre +'_avis.csv'
  print('Load : ', file_path)
  df = pd.read_csv(file_path)
  return df

In [12]:
df_sonic = get_dataframe('Sonic2')
df_sonic.head()

Load :  /content/drive/MyDrive/ISEN/Projets/NLP-Critiques_Films/Datasets/allocine_Sonic2_avis.csv


,Note,Description,key
0,"4,0",une bonne suite pour les aventures de sonic au...,0
1,"5,0",Cette suite de sonic est incroyable !Le fan qu...,1
2,"3,0",Malgré deux nouveaux personnages de l'univers ...,2
3,"4,0",Très bon film ont retrouve ce qui fait l’espri...,3
4,"4,0","Super film de ouf, le fait que Knuckles soit d...",4


In [13]:
import os
from tqdm import tqdm

def import_data(rootFolderPath):
    data = []
    data_label = []
    total = 0
    
    for root, dirs, files in os.walk(rootFolderPath):
        print(root, dirs, len(files))
        label = os.path.basename(root)
        total += len(files)
        for file in files:
            path = root+os.sep+file
            df = pd.read_csv(path)
            data.append(df)
            data_label.append(label)
            
    return(data, data_label)

def import_dataset(rootFolderPath):
    data = pd.DataFrame()
    total = 0
    
    for root, dirs, files in os.walk(rootFolderPath):
        print(root, dirs, len(files))
        label = os.path.basename(root)
        total += len(files)
        for file in files:
            path = root+os.sep+file
            df = pd.read_csv(path)
            data = pd.concat([data, df], ignore_index=True)
    return data


df = import_dataset(root_path+os.sep+"Datasets")

#data_test, data_test_label = import_data(root_path+os.sep+"Datasets")

#data_train, data_train_label = import_data("."+os.sep+"Dataset"+os.sep+"training")

#data_test_label = list(map(int, data_test_label))
#data_train_label = list(map(int, data_train_label))
df

/content/drive/MyDrive/ISEN/Projets/NLP-Critiques_Films/Datasets [] 2


,Note,Description,key
0,"4,0",une bonne suite pour les aventures de sonic au...,0
1,"5,0",Cette suite de sonic est incroyable !Le fan qu...,1
2,"3,0",Malgré deux nouveaux personnages de l'univers ...,2
3,"4,0",Très bon film ont retrouve ce qui fait l’espri...,3
4,"4,0","Super film de ouf, le fait que Knuckles soit d...",4
5,"2,5",Bon alors je trouve que la transposition de l'...,5
6,"1,5",Une suite uniquement tournée vers les plus jeu...,6
7,"4,0","Grand fan de Sonic, j'ai fait quelques kilomèt...",7
8,"4,0","Super à voir en famille, parent enfants, aucun...",8
9,"5,0",Franchement la presse abuse sur la note j'ai e...,9


## Etape 2 : Préparation des données

Ayant maintenant nos jeux de données, il faut les préparer afin de pouvoir modéliser notre analyse de sentiments. Pour cela nous allons faire appel à plusieurs techniques :

- Des expressions régulières pour retirer les bruits (ponctuation, etc.) des commentaires.
- Du NLP pour tokeniser et réduire le corpus de chaque commentaire (afin par exemple de ne garder que les mots importants via les stopwords)
- Des sacs de mots afin de « transformer » nos mots en nombres qui pourront alors être exploités dans un algorithme de Machine learning

_Les commentaires sont maintenant filtrés à leur essentiel._


Pre-processing :
- Supprimer les caractères spéciaux
- Remplacer les retours à la ligne (`\n` et `<br>`) par des espaces
- Mise en minuscules des caractères

In [14]:
import re
REMPLACE_SANS_ESPACE = re.compile("[;:!\'?,\"()\[\]]")
REMPLACE_AVEC_ESPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)|[.]")

def preprocess(txt):
    txt = [line.replace('\n', ' ')  for line in txt] # Retire les \n
    txt = [REMPLACE_SANS_ESPACE.sub("", line.lower()) for line in txt]
    txt = [REMPLACE_AVEC_ESPACE.sub(" ", line) for line in txt]

    return txt


In [15]:
df['Description'] = pd.DataFrame(preprocess(df['Description']))

df

,Note,Description,key
0,"4,0",une bonne suite pour les aventures de sonic au...,0
1,"5,0",cette suite de sonic est incroyable le fan que...,1
2,"3,0",malgré deux nouveaux personnages de lunivers d...,2
3,"4,0",très bon film ont retrouve ce qui fait l’espri...,3
4,"4,0",super film de ouf le fait que knuckles soit du...,4
5,"2,5",bon alors je trouve que la transposition de lu...,5
6,"1,5",une suite uniquement tournée vers les plus jeu...,6
7,"4,0",grand fan de sonic jai fait quelques kilomètre...,7
8,"4,0",super à voir en famille parent enfants aucun d...,8
9,"5,0",franchement la presse abuse sur la note jai eu...,9


### NLP

Import des bibliothèques

In [16]:
import nltk
nltk.download()
# Choisir :
# - (d) download
# - Identifier> all

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all
       | 
       | Downloading package abc to /root/nltk_data...
       |   Unzipping corpora/abc.zip.
       | Downloading package alpino to /root/nltk_data...
       |   Unzipping corpora/alpino.zip.
       | Downloading package averaged_perceptron_tagger to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger.zip.
       | Downloading package averaged_perceptron_tagger_ru to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger_ru.zip.
       | Downloading package basque_grammars to /root/nltk_data...
       |   Unzipping grammars/basque_grammars.zip.
       | Downloading package biocreative_ppi to /root/n

True

Import des bibliothèques

In [17]:
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.tokenize import sent_tokenize
import re

Définition des *stop words*

In [18]:
french_stopwords = set(stopwords.words('french'))
filtre_stopfr =  lambda text: [token for token in text if token.lower() not in french_stopwords]

def tokenize_word(desc):
  wt = word_tokenize(desc, language="french")
  fst = filtre_stopfr(wt)
  return fst

for desc in df['Description']:
  print(desc, "\n--->", tokenize_word(desc))

#print(word_tokenize(str(df['Description']), language="french"))
#print(filtre_stopfr(word_tokenize(str(df['Description']), language="french")))

une bonne suite pour les aventures de sonic au cinéma deja le premier film était super bienun bon divertissement pour toute la famille avec un jim carrey en pleine formeune suite explosive trop cool 
---> ['bonne', 'suite', 'aventures', 'sonic', 'cinéma', 'deja', 'premier', 'film', 'super', 'bienun', 'bon', 'divertissement', 'toute', 'famille', 'jim', 'carrey', 'pleine', 'formeune', 'suite', 'explosive', 'trop', 'cool']
cette suite de sonic est incroyable le fan que je suis est ravi en 1992 quand j y jouais sur megadrive on m aurait montré ce film je serais devenu dingue l univers du jeu est parfaitement respecté avec amour et passion  spoiler   tails et son avion knuckle robotnik et ses robot guêpes sonic au fond de l eau qui absorbe des bulles d oxygene les émeraudes les clins d œil à droite à gauche qui rendent hommage au jeu et bon sang la transformation en super sonic  juste magnifique   lhistoire est toujours aussi sympathique avec sonic et sa petite famille adoptive et jim carre

In [19]:
df['Tokens'] = df['Description'].apply(tokenize_word)

df

,Note,Description,key,Tokens
0,"4,0",une bonne suite pour les aventures de sonic au...,0,"[bonne, suite, aventures, sonic, cinéma, deja,..."
1,"5,0",cette suite de sonic est incroyable le fan que...,1,"[cette, suite, sonic, incroyable, fan, ravi, 1..."
2,"3,0",malgré deux nouveaux personnages de lunivers d...,2,"[malgré, deux, nouveaux, personnages, lunivers..."
3,"4,0",très bon film ont retrouve ce qui fait l’espri...,3,"[très, bon, film, retrouve, fait, ’, esprit, s..."
4,"4,0",super film de ouf le fait que knuckles soit du...,4,"[super, film, ouf, fait, knuckles, côté, deggm..."
5,"2,5",bon alors je trouve que la transposition de lu...,5,"[bon, alors, trouve, transposition, lunivers, ..."
6,"1,5",une suite uniquement tournée vers les plus jeu...,6,"[suite, uniquement, tournée, vers, plus, jeune..."
7,"4,0",grand fan de sonic jai fait quelques kilomètre...,7,"[grand, fan, sonic, jai, fait, quelques, kilom..."
8,"4,0",super à voir en famille parent enfants aucun d...,8,"[super, voir, famille, parent, enfants, aucun,..."
9,"5,0",franchement la presse abuse sur la note jai eu...,9,"[franchement, presse, abuse, note, jai, frisso..."


### Tokenisation

In [20]:
def tokenize_sent(data):
  st = sent_tokenize(data, language="french")
  wt = word_tokenize(data, language="french")
  phfr = filtre_stopfr( word_tokenize(data, language="french") )
  return phfr

for desc in df['Description']:
  print("-W->", tokenize_word(desc))
  print("-S->", tokenize_sent(desc))




-W-> ['bonne', 'suite', 'aventures', 'sonic', 'cinéma', 'deja', 'premier', 'film', 'super', 'bienun', 'bon', 'divertissement', 'toute', 'famille', 'jim', 'carrey', 'pleine', 'formeune', 'suite', 'explosive', 'trop', 'cool']
-S-> ['bonne', 'suite', 'aventures', 'sonic', 'cinéma', 'deja', 'premier', 'film', 'super', 'bienun', 'bon', 'divertissement', 'toute', 'famille', 'jim', 'carrey', 'pleine', 'formeune', 'suite', 'explosive', 'trop', 'cool']
-W-> ['cette', 'suite', 'sonic', 'incroyable', 'fan', 'ravi', '1992', 'quand', 'jouais', 'megadrive', 'montré', 'film', 'devenu', 'dingue', 'univers', 'jeu', 'parfaitement', 'respecté', 'amour', 'passion', 'spoiler', 'tails', 'avion', 'knuckle', 'robotnik', 'robot', 'guêpes', 'sonic', 'fond', 'eau', 'absorbe', 'bulles', 'oxygene', 'émeraudes', 'clins', 'œil', 'droite', 'gauche', 'rendent', 'hommage', 'jeu', 'bon', 'sang', 'transformation', 'super', 'sonic', 'juste', 'magnifique', 'lhistoire', 'toujours', 'aussi', 'sympathique', 'sonic', 'petite',

### Fréquence de distribution des valeurs

In [21]:
words = []

for ph in df['Tokens']:
  print("P: ", ph)
  for word in ph:
    words.append(word)

#words = str(words)
print(words)

fd = nltk.FreqDist(words)
for fword in fd.most_common():
  print(fword)

P:  ['bonne', 'suite', 'aventures', 'sonic', 'cinéma', 'deja', 'premier', 'film', 'super', 'bienun', 'bon', 'divertissement', 'toute', 'famille', 'jim', 'carrey', 'pleine', 'formeune', 'suite', 'explosive', 'trop', 'cool']
P:  ['cette', 'suite', 'sonic', 'incroyable', 'fan', 'ravi', '1992', 'quand', 'jouais', 'megadrive', 'montré', 'film', 'devenu', 'dingue', 'univers', 'jeu', 'parfaitement', 'respecté', 'amour', 'passion', 'spoiler', 'tails', 'avion', 'knuckle', 'robotnik', 'robot', 'guêpes', 'sonic', 'fond', 'eau', 'absorbe', 'bulles', 'oxygene', 'émeraudes', 'clins', 'œil', 'droite', 'gauche', 'rendent', 'hommage', 'jeu', 'bon', 'sang', 'transformation', 'super', 'sonic', 'juste', 'magnifique', 'lhistoire', 'toujours', 'aussi', 'sympathique', 'sonic', 'petite', 'famille', 'adoptive', 'jim', 'carrey', 'incroyable', 'eggman', 'film', 'merveille', 'vivement', 'suite']
P:  ['malgré', 'deux', 'nouveaux', 'personnages', 'lunivers', 'jeu', 'vidéo', 'sonic', 'knuckles', 'tails', 'assez', 'l

### Stemmatisation

Regrouper les mots ayant la même racine synthaxique

# WIP

In [22]:
example_words = ["donner","don","donne","donnera","dons","test"]
#stemmer = FrenchStemmer()
stemmer=nltk.stem.SnowballStemmer('french')
for w in example_words:
    print(stemmer.stem(w))

don
don
don
don
don
test


In [23]:
phfrlist = [stemmer.stem(x) for x in phfr]
phfrlist

NameError: ignored

### Etape 3 : Préparation des libellés

Jusque là, à chaque commentaire est associé une note de 1 à 5 et non une classe binaire. Il nous faut donc convertir nos notes en : 1 pour avis positif et 0 : pour avis négatif

_Note: N’oublions pas à la fin de retirer la note du jeu de données._


### Etape 4 : Finalisation de nos jeux de données

Les données sont presque prêtes mais nos commentaires qui sont maintenant sous forme de sac de mots doivent être convertis en nombre. Pour cela, il va falloir vectoriser nos mots (technique des sacs de mots) :

Vous devriez avoir maintenant une belle matrice avec beaucoup de colonnes (qui correspond au nombre de mots du corpus)

Afficher deux \_WordCloud \_: le WordCloud des avis positifs et celui des avis négatifs.


### Etape 5 : Entraînement du modèle

Nos données sont prêtes, nous allons pour ce premier exercice utiliser un algorithme de Regression Logistique comme ici il est question de classification binaire. Entraînons le modèle maintenant, et regardons sa précision par rapport au libellés connus.


### Etape 6 : Analyse des résultats

Calculer l’accuracy et la matrice de confusion sur les données de test. Une fois que les résultats sont satisfaisants, vous pourrez maintenant tester sur des commentaires que vous et vos collègues ferons afin de vérifier le bon fonctionnement du programme.